In [8]:
import tensorflow as tf # pip install tensorflow

# Hide GPU from visible devices for tensorflow 2.3.1 and above
tf.config.set_visible_devices([], 'GPU')

from tensorflow import keras
from tensorflow.keras import layers
import cv2 # pip install opencv-python
#pip install opencv-contrib-python    full package
import os
import matplotlib.pyplot as plt # pip install matplotlib
import numpy as np # pip install numpy
import random

In [7]:
# LOADING ALL IMAGES AND ENHANCING THEM
Datadirectory = "Training/"

Classes = ["0","1","2","3","4","5","6"]

training_Data = [] # data array

img_size = 224

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category) # 0, 1, label
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_training_Data()
# print(len(training_Data)) # to view numbers of training images
# type(training_Data) # to view type
# temp = np.array(training_Data) # change from list to array
# temp.shape # to view the shape of the images

random.shuffle(training_Data) # shuffling the data set

X = [] # data/feature
y = [] # label

for features,label in training_Data:
    X.append(features)
    y.append(label)

# converting images into 4D and change from list to array
X = np.array(X).reshape(-1, img_size, img_size, 3)
# X.shape # to check the shape of X

# normalizing images before using deep learning
# X = X/255.0; for computer with better RAM
for i in range(0, 28709): #numbers of images
    X[i] = X[i]/255.0
    
Y = np.array(y)
# Y.shape # must have the same numbers as X


In [ ]:
# DEEP LEARNING MODEL FOR TRAINING - TRANSFER LEARNING

model = tf.keras.applications.MobileNetV2() # Pre-trained Model
# model.summary() # check the layers of model

# Transfer learning - tuning, weights will start from last check point
# take 1st layer of the pre-trained model as input
base_input = model.layers[0].input
# take 2nd last layer of the pre-trained model as output
base_output = model.layers[-2].output
# base_output # to view shape and layer of the output

# adding new layer, after the output of global pooling layer
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output) # activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)
# for 7 classes # using softmax classification layer
# final_output # to view shape and layer of the final output

# creating new model
new_model = keras.Model(inputs = base_input, outputs = final_output)
# new_model.summary() # to check layers in new model

new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# new_model.fit(X, Y, epochs=25) # to train new model
# new_model.save('finalModel.h5') # to save the trained model in h5 format
# new_evaluate # to test data and accuracy
# new_model = tf.keras.models.load_model('finalModel.h5') # to load the trained model